
# BASIC CONFIGURATIONS

In [6]:
# Download NLTK data
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download essential NLTK resources
print("Downloading NLTK data...")

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

# Try to download wordnet - used for lemmatization
try:
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    lemmatization_available = True
    print("WordNet successfully loaded - lemmatization available")
except:
    lemmatization_available = False
    print("WordNet could not be loaded - proceeding without lemmatization")

WordNet successfully loaded - lemmatization available


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\karth\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\karth\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karth\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\karth\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
import os
from dotenv import load_dotenv

# Create and load .env file if it doesn't exist
if not os.path.exists('.env'):
    with open('.env', 'w') as f:
        f.write("# Environment variables for models\n")
        f.write("TRANSFORMERS_HOME=./models\n")
        f.write("CHROMADB_PATH=./chroma_db\n")

load_dotenv()

os.makedirs('./models', exist_ok=True)
os.makedirs('./chroma_db', exist_ok=True)

print("environment variables configured.")

environment variables configured.


In [8]:
import pandas as pd
import dateparser
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

In [9]:

dataset_path = 'SMData.xlsx'

try:
    df = pd.read_excel(dataset_path)
    print(f"Dataset loaded successfully with {df.shape[0]} rows and {df.shape[1]} columns.")
    print("\nDataset columns:")
    for col in df.columns:
        print(f"- {col}")
    print("\nSample data:")
    display(df.head(3))
except FileNotFoundError:
    print(f"Error: File '{dataset_path}' not found.")
    

Error: File 'SMData.xlsx' not found.


In [10]:
print("Verifying installations and initializing models...")

# Initialize embedding model
try:
    model_name = 'all-MiniLM-L6-v2'  # Small but effective model for embeddings
    embedding_model = SentenceTransformer(model_name)
    print(f"Sentence Transformer model '{model_name}' loaded successfully.")
except Exception as e:
    print(f"Error loading embedding model: {str(e)}")

# Initialize ChromaDB client
try:
    # Using PersistentClient for local storage
    chroma_client = chromadb.PersistentClient(path="./chroma_db", settings=Settings())
    print("ChromaDB client initialized successfully.")
except Exception as e:
    print(f"Error initializing ChromaDB client: {str(e)}")
    
print("\nEnvironment setup complete!")


Verifying installations and initializing models...
Sentence Transformer model 'all-MiniLM-L6-v2' loaded successfully.
ChromaDB client initialized successfully.

Environment setup complete!


In [11]:
import platform
import sys

# Verify AutoGen is working
try:
    from autogen import AssistantAgent, UserProxyAgent
    print("AutoGen package loaded successfully.")
except Exception as e:
    print(f"Error loading AutoGen: {str(e)}")


# Display system information - helpful for debugging

print(f"Python version: {platform.python_version()}")
print(f"Operating System: {platform.system()} {platform.release()}")
print(f"Current date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\nAll systems ready for the Agentic RAG implementation!")


AutoGen package loaded successfully.
Python version: 3.11.0
Operating System: Windows 10
Current date: 2025-05-12 21:46:57

All systems ready for the Agentic RAG implementation!



# Data Preprocessing

In [12]:
import re

text_columns = ['Next Steps', 'Outcome of meeting']


# Text cleaning function with fallback options
def clean_text(text, use_lemmatizer=True):
    """Clean and normalize text data"""
    if pd.isna(text) or text == "":
        return ""
    
    # Lowercase
    text = str(text).lower()
    
    # Remove special characters and digits
    text = re.sub(r'[^a-z\s]', ' ', text)
    
    # Tokenize
    try:
        tokens = word_tokenize(text)
    except Exception as e:
        print(f"Tokenization error: {e}. Using basic split instead.")
        tokens = text.split()
    
    # Remove stopwords if available
    try:
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]
    except Exception as e:
        print(f"Stopwords not available: {e}. Skipping stopword removal.")
    
    # Lemmatize if available and requested
    if lemmatization_available and use_lemmatizer:
        try:
            tokens = [lemmatizer.lemmatize(word) for word in tokens]
        except Exception as e:
            print(f"Lemmatization error: {e}. Using original tokens.")
    
    # Join back to string
    return ' '.join(tokens)


In [13]:
# Function to clean text columns
def process_text_columns(df):
    """Clean text data in specified columns"""
    text_columns = ['Next Steps', 'Outcome of meeting']
    for col in text_columns:
        if col in df.columns:
            print(f"Cleaning text in {col}...")
            df[col] = df[col].apply(lambda x: clean_text(x, use_lemmatizer=lemmatization_available))
    return df


In [14]:
# Function to extract key entities
def extract_entities(df):
    """Extract key entities like customer names and products"""
    entities = {}
    
    # Extract customer names
    if 'Visit Plan: Owner Name Customer' in df.columns:
        entities['customers'] = df['Visit Plan: Owner Name Customer'].dropna().unique().tolist()
    elif 'Customer' in df.columns:
        entities['customers'] = df['Customer'].dropna().unique().tolist()
    else:
        entities['customers'] = []
    
    # Extract products
    if 'Visit Plan: Product Division' in df.columns:
        entities['products'] = df['Visit Plan: Product Division'].dropna().unique().tolist()
    else:
        entities['products'] = []
    
    print(f"Extracted {len(entities['customers'])} unique customers and {len(entities['products'])} unique products.")
    return entities


In [15]:
# Function to handle missing values
def handle_missing_values(df):
    """Handle missing values in the DataFrame"""
    # Fill missing values in text columns with empty string
    text_columns = ['Next Steps', 'Outcome of meeting']
    for col in text_columns:
        if col in df.columns:
            df[col] = df[col].fillna("")
    
    # Fill missing values in categorical columns with 'Unknown'
    categorical_columns = ['Visit Plan: Owner Region', 'Visit Plan: Product Division']
    for col in categorical_columns:
        if col in df.columns:
            df[col] = df[col].fillna("Unknown")
    
    return df


In [16]:
# Function to standardize formats
def standardize_formats(df):
    """Standardize formats across columns"""
    # Strip whitespace from string columns
    for col in df.select_dtypes(include=['object']).columns:
        if hasattr(df[col], 'str'):  # Check if it's a string column
            df[col] = df[col].str.strip()
    
    return df


In [17]:
# Main preprocessing function
def preprocess_data(df):
    """Main function to preprocess the data"""
    print("Starting data preprocessing...")
    
    # Make a copy to avoid modifying the original dataframe
    df_processed = df.copy()
    
    # Apply preprocessing steps
    # df_processed = convert_dates(df_processed)
    df_processed = process_text_columns(df_processed)
    entities = extract_entities(df_processed)
    df_processed = handle_missing_values(df_processed)
    df_processed = standardize_formats(df_processed)
    
    print("Data preprocessing completed successfully.")
    print (f"Processed DataFrame shape: {df_processed.shape}")
    print("Sample processed data:")
    print(df_processed.head())
    
    return df_processed, entities


In [18]:
try:
    df
except NameError:
    print("Error: DataFrame 'df' not found.")

# Process the data
df_processed, entities = preprocess_data(df)


Error: DataFrame 'df' not found.


NameError: name 'df' is not defined


# Vectorization & Embedding Creation

In [ ]:
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
import torch
import numpy as np

print("Setting up vectorization and embedding creation pipeline...")

print("Loading the sentence transformer model...")

# all-MiniLM-L6-v2 - maps sentences to a 384-dimensional dense vector space
model_name = 'all-MiniLM-L6-v2'

try:
    # Check if CUDA is available for faster processing
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"  - Using device: {device}")
    
    embedding_model = SentenceTransformer(model_name, device=device)
    embedding_dimension = embedding_model.get_sentence_embedding_dimension()
    print(f"  - Successfully loaded '{model_name}' model")
    print(f"  - Embedding dimension: {embedding_dimension}")
except Exception as e:
    print(f"  - Error loading model: {str(e)}")
    print("  - Falling back to a simpler approach...")
    embedding_dimension = 384  # Default


In [ ]:
# combine text fields into a single representation
def create_combined_text(row, text_columns=['Next Steps', 'Outcome of meeting'], 
                         metadata_columns=['Visit Plan: Owner Name', 
                                          'Customer', 
                                          'Visit Plan: Product Division', 'Customer SAP Code', 'Visit Plan: Visit Date',
                                          'Visit Report: Created Date',	'Visit Plan: Owner Region',	'Visit Plan: Visit Owner Email']):

    """
    Create a combined text representation from multiple columns in a row.
    This is to prioritize text columns while adding context from metadata columns.
    """
    text_parts = []
    
    # Add text from main content columns
    for col in text_columns:
        if col in row and pd.notna(row[col]) and row[col]:
            text_parts.append(f"{col}: {row[col]}")
    
    # Add context from metadata columns
    metadata_parts = []
    for col in metadata_columns:
        if col in row and pd.notna(row[col]) and row[col]:
            metadata_parts.append(f"{col}: {row[col]}")
    
    combined_text = " ".join(text_parts)
    
    # Add metadata with a separator if we have both text and metadata
    if text_parts and metadata_parts:
        combined_text += " | " + " | \n".join(metadata_parts)
    elif metadata_parts:
        combined_text = " | ".join(metadata_parts)
    
    return combined_text

# Testing for a sample row
if not df.empty:
    sample_combined = create_combined_text(df.iloc[0])
    print("Creating combined text representations:")
    print(f"  - Sample combined text (truncated): {sample_combined}")


In [ ]:
# Generate embeddings for each row
print("\nGenerating embeddings for each row...")

# Create a new column for the combined text
df['combined_text'] = df.apply(create_combined_text, axis=1)

# Function to generate embeddings in batches
def generate_embeddings_in_batches(texts, model, batch_size=32):
    """Generate embeddings for a list of texts in batches to avoid memory issues"""
    all_embeddings = []
    
    # Process in batches
    for i in tqdm(range(0, len(texts), batch_size), desc="Generating embeddings"):
        batch_texts = texts[i:i+batch_size]
        batch_embeddings = model.encode(batch_texts, show_progress_bar=False, 
                                        normalize_embeddings=True)
        all_embeddings.extend(batch_embeddings)
    
    return np.array(all_embeddings)

# Generate embeddings
try:
    # Get the list of combined texts
    texts = df['combined_text'].tolist()
    
    # Generate embeddings
    embeddings = generate_embeddings_in_batches(texts, embedding_model)
    
    # Store embeddings in the dataframe
    df['embedding'] = list(embeddings)
    
    print(f"  - Successfully generated {len(embeddings)} embeddings")
    print(f"  - Embedding shape: {embeddings[0].shape}")
    
    # Show a sample embedding (first 5 dimensions)
    if not df.empty:
        sample_embedding = df['embedding'].iloc[0]
        print(f"  - Sample embedding (first 5 dimensions): {sample_embedding[:5]}")
except Exception as e:
    print(f"  - Error generating embeddings: {str(e)}")


In [ ]:
# Create metadata for each embedding
print("Creating metadata for each embedding...")

def create_metadata(row):
    """Create a metadata dictionary for a row"""
    metadata = {}
    
    # Add date information
    for date_col in ['Visit Plan: Visit Date', 'Visit Report: Created Date']:
        if date_col in row and pd.notna(row[date_col]):
            metadata[date_col.replace(':', '_')] = str(row[date_col])
    
    # Add key identifying information
    for col in ['Visit Plan: Visit Owner Email', 'Customer', 'Visit Plan: Product Division', 
                'Visit Plan: Owner Region', 'Customer SAP Code', 'Visit Plan: Owner Name']:
        if col in row and pd.notna(row[col]):
            metadata[col.replace(':', '_')] = str(row[col])
    
    # Add the original text content
    for col in ['Next Steps', 'Outcome of meeting']:
        if col in row and pd.notna(row[col]):
            metadata[col.replace(':', '_')] = str(row[col])
    
    # Add the combined text used for embedding
    metadata['combined_text'] = row['combined_text']
    
    return metadata

# Create metadata for each row
df['metadata'] = df.apply(create_metadata, axis=1)

# Show a sample metadata
if not df.empty:
    sample_metadata = df['metadata'].iloc[0]
    print("  - Sample metadata:")
    for key, value in list(sample_metadata.items()):  
        print(f"    - {key}: {value[:50]}..." if isinstance(value, str) and len(value) > 50 else f"    - {key}: {value}")


In [ ]:
import chromadb
from chromadb.utils import embedding_functions

print("Setting up ChromaDB for vector storage...")

os.makedirs("./chroma_db", exist_ok=True)

try:
    # Initialize ChromaDB client
    chroma_client = chromadb.PersistentClient(path="./chroma_db")
    
    # Create an embedding function that matches our model
    embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=model_name)
    
    collection_name = "customer_visits"
    
    # Delete collection if it already exists (for clean restart)
    try:
        chroma_client.delete_collection(collection_name)
        print(f"  - Deleted existing collection: {collection_name}")
    except:
        pass
    
    # Create a new collection
    collection = chroma_client.create_collection(
        name=collection_name,
        embedding_function=embedding_function,
        metadata={"description": "Customer visit data with embeddings"}
    )
    
    print(f"  - Successfully created ChromaDB collection: {collection_name}")
except Exception as e:
    print(f"  - Error setting up ChromaDB: {str(e)}")


In [ ]:
print("Storing embeddings and metadata in ChromaDB...")

try:
    # Prepare data for batch addition
    ids = []
    documents = []
    metadatas = []
    embeddings_list = []
    
    # Process each row
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Preparing data"):
        # Create a unique ID
        unique_id = f"doc_{idx}"
        
        # Get the combined text
        document = row['combined_text']
        
        # Get the metadata
        metadata = row['metadata']
        
        # Get the embedding
        embedding = row['embedding']
        
        # Add to lists
        ids.append(unique_id)
        documents.append(document)
        metadatas.append(metadata)
        embeddings_list.append(embedding)
    
    # Add data to ChromaDB in batches
    batch_size = 100
    for i in tqdm(range(0, len(ids), batch_size), desc="Adding to ChromaDB"):
        batch_ids = ids[i:i+batch_size]
        batch_documents = documents[i:i+batch_size]
        batch_metadatas = metadatas[i:i+batch_size]
        batch_embeddings = embeddings_list[i:i+batch_size]
        
        # Add to collection
        collection.add(
            ids=batch_ids,
            documents=batch_documents,
            metadatas=batch_metadatas,
            embeddings=batch_embeddings
        )
    
    # Get collection count to verify
    count = collection.count()
    print(f"  - Successfully stored {count} embeddings in ChromaDB")
except Exception as e:
    print(f"  - Error storing embeddings in ChromaDB: {str(e)}")


In [ ]:
print("Testing a simple query to verify the embeddings...")

try:
    # Create a test query
    test_query = "what will rinac india do with the overdue?"
    
    # Query the collection
    results = collection.query(
        query_texts=[test_query],
        n_results=2,
        include=["documents", "metadatas", "distances"]
    )
    
    # Display results
    print(f"Query: '{test_query}'\n")
    print(f"Found {len(results['documents'][0])} results\n")
    
    for i, (doc, metadata, distance) in enumerate(zip(
        results['documents'][0], 
        results['metadatas'][0], 
        results['distances'][0]
    )):
        print(f"Result {i+1} (similarity: {1-distance:.4f}):")
        print(f"Document: {doc[:100]}...")
        print(f"Customer: {metadata.get('Customer', 'N/A')}")
        print(f"Product: {metadata.get('Visit Plan_ Product Division', 'N/A')}")
        # print(f"Visit Plan: Owner Name: {metadata.get('Visit Plan_ Owner Name', 'N/A')}")
        # print(f"Outcome of meeting: {metadata.get('Outcome of meeting', 'N/A')}")
        # print(f"Visit Plan: Customer SAP: {metadata.get('Customer SAP Code', 'N/A')}")
        # print(f"Visit Plan: Owner Region: {metadata.get('Visit Plan_ Owner Region', 'N/A')}")
        # print(f"Visit Plan: Visit Owner Email: {metadata.get('Visit Plan_ Visit Owner Email', 'N/A')}")
        # print(f"Visit Report: Created Date: {metadata.get('Visit Report_ Created Date\n', 'N/A\n' )}")

except Exception as e:
    print(f"  - Error testing query: {str(e)}")


In [ ]:
print("Saving processed data...")

try:
    # Save the dataframe without the embedding column (to save space)
    df_to_save = df.drop(columns=['embedding'])
    
    df_to_save.to_csv("processed_customer_visits.csv", index=False)
    print("Saved processed data to 'processed_customer_visits.csv'")
    
    import json
    
    # Create a mapping of document IDs to their metadata
    metadata_mapping = {f"doc_{idx}": row['metadata'] for idx, row in df.iterrows()}
    
    # Save to JSON
    with open("document_metadata.json", "w") as f:
        json.dump(metadata_mapping, f)
    
    print("Saved metadata mapping to 'document_metadata.json'")
except Exception as e:
    print(f"Error saving data: {str(e)}")

print("Vectorization and embedding creation complete!")


# Agentic Workflow

In [ ]:
# Import required libraries for the agent framework
import asyncio
from typing import Dict, List, Any, Optional
import json

# Import AutoGen components
from autogen import AssistantAgent
from autogen import UserMessage
from autogen import OllamaChatCompletionClient

# For vector database operations
import chromadb


In [ ]:
def get_ollama_config():
    """
    Creates configuration for Ollama LLM client
    
    Returns:
        OllamaChatCompletionClient: Configured Ollama client
    """
    # Create an Ollama client with a suitable model
    # You can change the model to any available in your Ollama installation
    model_client = OllamaChatCompletionClient(
        model="qwen2.5-coder:0.5b",  # Or any other model you have in Ollama
        temperature=0.7
    )
    
    return model_client


In [ ]:
def create_query_analyzer_agent(model_client):
    """
    Creates an agent that analyzes and extracts intent from user queries
    
    Args:
        model_client: The LLM client for the agent
        
    Returns:
        AssistantAgent: Configured query analyzer agent
    """
    system_message = """You are a Query Analyzer Agent specialized in understanding user queries about customer visit data.
    Your task is to:
    1. Identify the main intent of the query (e.g., customer information, visit outcomes, regional analysis)
    2. Extract key entities mentioned in the query (customers, regions, dates, products)
    3. Determine time ranges if mentioned (specific dates, months, quarters)
    4. Identify any filtering conditions (e.g., specific regions, product divisions)
    5. Format the extracted information in a structured way for the retrieval agent
    
    Respond with a JSON object containing:
    - intent: The primary purpose of the query
    - entities: Key entities mentioned
    - time_range: Any time periods mentioned
    - filters: Any filtering conditions
    - search_terms: Key terms for vector search
    """
    
    return AssistantAgent(
        name="QueryAnalyzer",
        model_client=model_client,
        system_message=system_message
    )

def create_data_retriever_agent(model_client, collection):
    """
    Creates an agent that retrieves relevant data from the vector database
    
    Args:
        model_client: The LLM client for the agent
        collection: ChromaDB collection for vector search
        
    Returns:
        AssistantAgent: Configured data retriever agent
    """
    system_message = """You are a Data Retriever Agent specialized in finding relevant customer visit information.
    Your task is to:
    1. Use the structured query information to search the vector database
    2. Apply any filters to narrow down results
    3. Retrieve the most relevant entries
    4. Format the retrieved data in a clear, structured way
    5. Include metadata about the search results (total found, relevance)
    
    The vector database contains customer visit reports with the following fields:
    - Visit dates
    - Customer information
    - Product divisions
    - Regional data
    - Visit outcomes
    - Next steps
    
    Respond with a JSON object containing:
    - retrieved_data: The relevant entries found
    - metadata: Information about the search results
    - suggested_focus: Areas that might be most relevant to the query
    """
    
    async def search_vector_db(query_info):
        """Tool to search the vector database based on query information"""
        try:
            # Extract search terms from query info
            search_terms = query_info.get("search_terms", [])
            if not search_terms:
                return {"error": "No search terms provided"}
            
            # Combine search terms into a query string
            query_string = " ".join(search_terms) if isinstance(search_terms, list) else search_terms
            
            # Perform the vector search
            results = collection.query(
                query_texts=[query_string],
                n_results=5  # Adjust as needed
            )
            
            # Format the results
            formatted_results = []
            if results and 'documents' in results and results['documents']:
                for i, doc in enumerate(results['documents'][0]):
                    # Get metadata if available
                    metadata = {}
                    if 'metadatas' in results and results['metadatas'] and i < len(results['metadatas'][0]):
                        metadata = results['metadatas'][0][i]
                    
                    # Format the entry
                    formatted_results.append({
                        "text": doc,
                        "metadata": metadata,
                        "distance": results['distances'][0][i] if 'distances' in results else None
                    })
            
            return {
                "retrieved_data": formatted_results,
                "metadata": {
                    "total_found": len(formatted_results),
                    "query": query_string
                }
            }
        except Exception as e:
            return {"error": f"Error searching vector database: {str(e)}"}
    
    retriever = AssistantAgent(
        name="DataRetriever",
        model_client=model_client,
        system_message=system_message,
        tools=[search_vector_db]
    )
    
    return retriever

def create_response_generator_agent(model_client):
    """
    Creates an agent that generates final responses based on retrieved data
    
    Args:
        model_client: The LLM client for the agent
        
    Returns:
        AssistantAgent: Configured response generator agent
    """
    system_message = """You are a Response Generator Agent specialized in creating informative responses about customer visit data.
    Your task is to:
    1. Use the retrieved data to create a comprehensive response
    2. Structure the information in a clear, readable format
    3. Highlight key insights and patterns
    4. Provide direct answers to the original query
    5. Include relevant details from the visit reports
    
    Make your responses:
    - Concise but informative
    - Well-structured with headings when appropriate
    - Focused on answering the original question
    - Supported by data from the retrieved documents
    
    If the data is insufficient to answer the query completely, acknowledge the limitations.
    """
    
    return AssistantAgent(
        name="ResponseGenerator",
        model_client=model_client,
        system_message=system_message
    )

def create_orchestrator_agent(model_client):
    """
    Creates an agent that orchestrates the workflow between other agents
    
    Args:
        model_client: The LLM client for the agent
        
    Returns:
        AssistantAgent: Configured orchestrator agent
    """
    system_message = """You are an Orchestrator Agent responsible for managing the workflow between specialized agents.
    Your task is to:
    1. Receive user queries and direct them to the Query Analyzer
    2. Take the structured query information and pass it to the Data Retriever
    3. Send retrieved data to the Response Generator
    4. Return the final response to the user
    5. Handle any errors or edge cases in the process
    
    Maintain the logical flow of information between agents and ensure each step is completed properly before proceeding to the next.
    """
    
    return AssistantAgent(
        name="Orchestrator",
        model_client=model_client,
        system_message=system_message
    )


In [ ]:
class AgentWorkflow:
    """
    Manages the workflow between agents for processing user queries
    """
    
    def __init__(self, collection):
        """
        Initialize the agent workflow
        
        Args:
            collection: ChromaDB collection for vector search
        """
        # Initialize the LLM client
        self.model_client = get_ollama_config()
        
        # Create the agents
        self.query_analyzer = create_query_analyzer_agent(self.model_client)
        self.data_retriever = create_data_retriever_agent(self.model_client, collection)
        self.response_generator = create_response_generator_agent(self.model_client)
        self.orchestrator = create_orchestrator_agent(self.model_client)
        
        print("Agent workflow initialized with all required agents")
    
    async def process_query(self, user_query):
        """
        Process a user query through the agent workflow
        
        Args:
            user_query: The user's natural language query
            
        Returns:
            str: The final response to the user query
        """
        try:
            print(f"Processing query: {user_query}")
            
            # Step 1: Analyze the query
            print("Step 1: Analyzing query...")
            analyzer_result = await self.query_analyzer.run(task=user_query)
            
            # Extract the structured query information
            try:
                query_info = json.loads(analyzer_result.content)
                print(f"Query analysis complete: {json.dumps(query_info, indent=2)}")
            except json.JSONDecodeError:
                # If the response is not valid JSON, use a simplified approach
                query_info = {"search_terms": user_query}
                print("Could not parse analyzer response as JSON. Using simplified query info.")
            
            # Step 2: Retrieve relevant data
            print("Step 2: Retrieving data...")
            retriever_message = f"Search for information related to: {json.dumps(query_info)}"
            retriever_result = await self.data_retriever.run(task=retriever_message)
            
            # Step 3: Generate the response
            print("Step 3: Generating response...")
            generator_message = f"""
            Original query: {user_query}
            
            Retrieved information: {retriever_result.content}
            
            Please generate a comprehensive response to the original query based on this information.
            """
            response_result = await self.response_generator.run(task=generator_message)
            
            print("Query processing complete")
            return response_result.content
            
        except Exception as e:
            error_message = f"Error processing query: {str(e)}"
            print(error_message)
            return f"I encountered an error while processing your query: {str(e)}"
    
    async def close(self):
        """Close the model client connection"""
        await self.model_client.close()


In [ ]:
def setup_agent_workflow(collection):
    """
    Set up the agent workflow
    
    Args:
        collection: ChromaDB collection for vector search
        
    Returns:
        AgentWorkflow: Configured agent workflow
    """
    return AgentWorkflow(collection)

async def main():
    """
    Main function to run the agent workflow
    """
    # Load the vector database
    print("Loading vector database...")
    client = chromadb.Client(Settings(persist_directory="./chroma_db"))
    collection = client.get_or_create_collection("customer_visits")
    
    # Set up the agent workflow
    print("Setting up agent workflow...")
    workflow = setup_agent_workflow(collection)
    
    print("Agent workflow ready!")
    print("\nYou can now ask questions about customer visit data.")
    
    # Simple interactive loop for testing
    try:
        while True:
            query = input("\nEnter your query (or 'exit' to quit): ")
            if query.lower() == 'exit':
                break
                
            response = await workflow.process_query(query)
            print("\nResponse:")
            print(response)
    finally:
        # Clean up resources
        await workflow.close()

# Run the main function if this script is executed directly
if __name__ == "__main__":
    main()


In [ ]:
def validate_collection(collection):
    """
    Validate that the ChromaDB collection is properly configured
    
    Args:
        collection: ChromaDB collection to validate
        
    Returns:
        bool: True if valid, False otherwise
    """
    try:
        # Check if the collection exists and has data
        count = collection.count()
        if count == 0:
            print("Warning: Vector database collection is empty")
            return False
        
        print(f"Vector database collection contains {count} documents")
        return True
    except Exception as e:
        print(f"Error validating collection: {str(e)}")
        return False

async def safe_agent_run(agent, task, max_retries=3):
    """
    Safely run an agent task with retry logic
    
    Args:
        agent: The agent to run
        task: The task to run
        max_retries: Maximum number of retry attempts
        
    Returns:
        Message: The agent's response
    """
    retries = 0
    while retries < max_retries:
        try:
            return await agent.run(task=task)
        except Exception as e:
            retries += 1
            print(f"Error running agent task (attempt {retries}/{max_retries}): {str(e)}")
            if retries >= max_retries:
                raise
            await asyncio.sleep(1)  # Brief pause before retrying


In [ ]:
class AgentWorkflow:
    """
    Manages the workflow between agents for processing user queries
    """
    
    def __init__(self, collection):
        """
        Initialize the agent workflow
        
        Args:
            collection: ChromaDB collection for vector search
        """
        # Validate the collection
        if not validate_collection(collection):
            print("Warning: Proceeding with potentially invalid collection")
        
        # Initialize the LLM client
        self.model_client = get_ollama_config()
        
        # Create the agents
        self.query_analyzer = create_query_analyzer_agent(self.model_client)
        self.data_retriever = create_data_retriever_agent(self.model_client, collection)
        self.response_generator = create_response_generator_agent(self.model_client)
        self.orchestrator = create_orchestrator_agent(self.model_client)
        
        print("Agent workflow initialized with all required agents")
    
    async def process_query(self, user_query):
        """
        Process a user query through the agent workflow
        
        Args:
            user_query: The user's natural language query
            
        Returns:
            str: The final response to the user query
        """
        try:
            print(f"Processing query: {user_query}")
            
            # Step 1: Analyze the query
            print("Step 1: Analyzing query...")
            analyzer_result = await safe_agent_run(self.query_analyzer, user_query)
            
            # Extract the structured query information
            try:
                query_info = json.loads(analyzer_result.content)
                print(f"Query analysis complete: {json.dumps(query_info, indent=2)}")
            except json.JSONDecodeError:
                # If the response is not valid JSON, use a simplified approach
                query_info = {"search_terms": user_query}
                print("Could not parse analyzer response as JSON. Using simplified query info.")
            
            # Step 2: Retrieve relevant data
            print("Step 2: Retrieving data...")
            retriever_message = f"Search for information related to: {json.dumps(query_info)}"
            retriever_result = await safe_agent_run(self.data_retriever, retriever_message)
            
            # Step 3: Generate the response
            print("Step 3: Generating response...")
            generator_message = f"""
            Original query: {user_query}
            
            Retrieved information: {retriever_result.content}
            
            Please generate a comprehensive response to the original query based on this information.
            """
            response_result = await safe_agent_run(self.response_generator, generator_message)
            
            print("Query processing complete")
            return response_result.content
            
        except Exception as e:
            error_message = f"Error processing query: {str(e)}"
            print(error_message)
            return f"I encountered an error while processing your query: {str(e)}"
    
    async def close(self):
        """Close the model client connection"""
        await self.model_client.close()


# Step 6: RAG Implementation


In [ ]:
# Import required libraries for RAG implementation
import numpy as np
from typing import List, Dict, Any, Optional, Union
from chromadb.config import Settings
import chromadb
from sentence_transformers import SentenceTransformer

# Define the embedding function that's compatible with ChromaDB's interface
class SentenceTransformerEmbedding:
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        """
        Initialize the embedding model
        
        Args:
            model_name: Name of the sentence transformer model to use
        """
        self.model = SentenceTransformer(model_name)
        
    def __call__(self, input: List[str]) -> List[List[float]]:
        """
        Generate embeddings for a list of texts
        
        Args:
            input: List of strings to embed
            
        Returns:
            List of embeddings as float arrays
        """
        embeddings = self.model.encode(input)
        return embeddings.tolist()


In [ ]:
def setup_vector_database(df, embedding_function=None):
    """
    Set up and populate the vector database with customer visit data
    
    Args:
        df: DataFrame containing customer visit data
        embedding_function: Function to generate embeddings
        
    Returns:
        chromadb.Collection: The populated collection
    """
    print("Setting up vector database...")
    
    # Initialize ChromaDB client
    chroma_path = os.getenv("CHROMADB_PATH", "./chroma_db")
    client = chromadb.Client(Settings(persist_directory=chroma_path))
    
    # Create or get the collection
    if embedding_function is None:
        embedding_function = SentenceTransformerEmbedding()
    
    # Check if collection exists and recreate if needed
    try:
        client.delete_collection("customer_visits")
        print("Existing collection deleted.")
    except:
        pass
    
    collection = client.create_collection(
        name="customer_visits",
        embedding_function=embedding_function
    )
    
    # Prepare documents for insertion
    documents = []
    metadatas = []
    ids = []
    
    for idx, row in df.iterrows():
        # Create a comprehensive document from the row data
        document = f"""
        Visit Date: {row['Visit Plan: Visit Date']}
        Created Date: {row['Visit Report: Created Date']}
        Region: {row['Visit Plan: Owner Region']}
        Owner Email: {row['Visit Plan: Visit Owner Email']}
        Owner Name: {row['Visit Plan: Owner Name']}
        Customer: {row['Customer']}
        Customer SAP Code: {row['Customer SAP Code']}
        Product Division: {row['Visit Plan: Product Division']}
        Next Steps: {row['Next Steps']}
        Outcome of meeting: {row['Outcome of meeting']}
        """
        
        # Create metadata for filtering
        metadata = {
            "visit_date": str(row['Visit Plan: Visit Date']),
            "created_date": str(row['Visit Report: Created Date']),
            "region": row['Visit Plan: Owner Region'],
            "owner_email": row['Visit Plan: Visit Owner Email'],
            "owner_name": row['Visit Plan: Owner Name'],
            "customer": row['Customer'],
            "customer_sap_code": str(row['Customer SAP Code']),
            "product_division": row['Visit Plan: Product Division']
        }
        
        documents.append(document)
        metadatas.append(metadata)
        ids.append(f"doc_{idx}")
    
    # Add documents to the collection in batches
    batch_size = 100
    for i in range(0, len(documents), batch_size):
        end_idx = min(i + batch_size, len(documents))
        collection.add(
            documents=documents[i:end_idx],
            metadatas=metadatas[i:end_idx],
            ids=ids[i:end_idx]
        )
        print(f"Added documents {i} to {end_idx-1}")
    
    print(f"Vector database setup complete with {len(documents)} documents")
    return collection

def load_vector_db():
    """
    Load the existing vector database
    
    Returns:
        chromadb.Collection: The loaded collection
    """
    print("Loading vector database...")
    
    # Try to get existing collection or create a new one
    try:
        collection = chroma_client.get_collection(
            name="customer_visits",
            embedding_function=embedding_function
        )
        count = collection.count()
        print(f"Loaded existing collection with {count} documents")
    except:
        print("Collection not found. Please run setup_vector_database first.")
        collection = chroma_client.create_collection(
            name="customer_visits",
            embedding_function=embedding_function
        )
    
    return collection


In [ ]:
class RAGProcessor:
    """
    Handles the Retrieval-Augmented Generation process
    """
    
    def __init__(self, collection):
        """
        Initialize the RAG processor
        
        Args:
            collection: ChromaDB collection for vector search
        """
        self.collection = collection
    
    def retrieve(self, query, n_results=5, filter_criteria=None):
        """
        Retrieve relevant documents based on the query
        
        Args:
            query: The query string
            n_results: Number of results to retrieve
            filter_criteria: Optional filter criteria for the query
            
        Returns:
            dict: Retrieved documents and their metadata
        """
        try:
            # Perform the vector search
            results = self.collection.query(
                query_texts=[query],
                n_results=n_results,
                where=filter_criteria
            )
            
            # Format the results
            formatted_results = []
            if results and 'documents' in results and results['documents']:
                for i, doc in enumerate(results['documents'][0]):
                    # Get metadata if available
                    metadata = {}
                    if 'metadatas' in results and results['metadatas'] and i < len(results['metadatas'][0]):
                        metadata = results['metadatas'][0][i]
                    
                    # Format the entry
                    formatted_results.append({
                        "text": doc,
                        "metadata": metadata,
                        "distance": results['distances'][0][i] if 'distances' in results else None
                    })
            
            return {
                "retrieved_data": formatted_results,
                "metadata": {
                    "total_found": len(formatted_results),
                    "query": query
                }
            }
        except Exception as e:
            print(f"Error in retrieval: {str(e)}")
            return {
                "error": f"Error retrieving data: {str(e)}",
                "retrieved_data": [],
                "metadata": {
                    "total_found": 0,
                    "query": query
                }
            }
    
    def augment_context(self, query, retrieved_data):
        """
        Augment the query with context from retrieved documents
        
        Args:
            query: The original query
            retrieved_data: Data retrieved from the vector database
            
        Returns:
            str: Augmented context for the LLM
        """
        if not retrieved_data or "error" in retrieved_data:
            return f"Query: {query}\n\nNo relevant information found in the database."
        
        # Extract the retrieved documents
        documents = retrieved_data.get("retrieved_data", [])
        if not documents:
            return f"Query: {query}\n\nNo relevant information found in the database."
        
        # Build the context
        context = f"Query: {query}\n\nRelevant information from customer visit database:\n\n"
        
        for i, doc in enumerate(documents):
            context += f"Document {i+1}:\n{doc['text']}\n\n"
        
        return context
    
    async def generate_response(self, augmented_context, model_client):
        """
        Generate a response using the LLM with augmented context
        
        Args:
            augmented_context: The context-augmented query
            
        Returns:
            str: Generated response
        """
        try:
            task_result = await self.response_agent.run(task=augmented_context)

            # Preferred: use last_message if available
            if hasattr(task_result, "last_message"):
                return task_result.last_message.content
            
            # Fallback: pull the last TextMessage from .messages
            if hasattr(task_result, "messages") and task_result.messages:
                # assume the last message is the assistant’s answer
                return task_result.messages[-1].content

            # In the unlikely case neither attribute exists
            return str(task_result)
        except Exception as e:
            return f"Error generating response: {str(e)}"



In [ ]:
def create_rag_enabled_data_retriever_agent(model_client, rag_processor):
    """
    Creates a RAG-enabled agent that retrieves relevant data
    
    Args:
        model_client: The LLM client for the agent
        rag_processor: The RAG processor instance
        
    Returns:
        AssistantAgent: Configured RAG-enabled data retriever agent
    """
    system_message = """You are a Data Retriever Agent specialized in finding relevant customer visit information.
    Your task is to:
    1. Analyze the query to identify key search terms
    2. Retrieve relevant information from the customer visit database
    3. Extract the most pertinent details from the retrieved documents
    4. Format the information in a clear, structured way
    
    The customer visit database contains reports with the following fields:
    - Visit dates
    - Customer information
    - Product divisions
    - Regional data
    - Visit outcomes
    - Next steps
    
    Respond with a comprehensive set of relevant information that addresses the query.
    """
    
    async def retrieve_data(query_info):
        """Tool to retrieve data from the vector database"""
        try:
            # Extract search terms and filters
            search_terms = query_info.get("search_terms", "")
            if not search_terms:
                return {"error": "No search terms provided"}
            
            # Extract any filter criteria
            filters = query_info.get("filters", {})
            
            # Convert filters to ChromaDB format if needed
            chroma_filters = {}
            if filters:
                for key, value in filters.items():
                    if key in ["region", "customer", "product_division", "owner_name"]:
                        chroma_filters[key] = value
            
            # Use RAG processor to retrieve relevant data
            results = rag_processor.retrieve(
                query=search_terms,
                n_results=5,
                filter_criteria=chroma_filters if chroma_filters else None
            )
            
            # Augment the context with retrieved data
            augmented_context = rag_processor.augment_context(search_terms, results)
            
            return {
                "augmented_context": augmented_context,
                "retrieved_data": results.get("retrieved_data", []),
                "metadata": results.get("metadata", {})
            }
        except Exception as e:
            return {"error": f"Error retrieving data: {str(e)}"}
    
    retriever = AssistantAgent(
        name="DataRetriever",
        model_client=model_client,
        system_message=system_message,
        tools=[retrieve_data]
    )
    
    return retriever


In [ ]:
class RAGEnabledWorkflow:
    """
    Manages the RAG-enabled workflow between agents for processing user queries
    """
    
    def __init__(self, collection):
        """
        Initialize the RAG-enabled agent workflow
        
        Args:
            collection: ChromaDB collection for vector search
        """
        # Validate the collection
        if not validate_collection(collection):
            print("Warning: Proceeding with potentially invalid collection")
        
        # Initialize the LLM client
        self.model_client = get_ollama_config()
        
        # Initialize the RAG processor
        self.rag_processor = RAGProcessor(collection)
        
        # Create the agents
        self.query_analyzer = create_query_analyzer_agent(self.model_client)
        self.data_retriever = create_rag_enabled_data_retriever_agent(self.model_client, self.rag_processor)
        self.response_generator = create_response_generator_agent(self.model_client)
        self.orchestrator = create_orchestrator_agent(self.model_client)
        
        print("RAG-enabled agent workflow initialized with all required agents")
    
    async def process_query(self, user_query):
        """
        Process a user query through the RAG-enabled agent workflow
        
        Args:
            user_query: The user's natural language query
            
        Returns:
            str: The final response to the user query
        """
        try:
            print(f"Processing query: {user_query}")
            
            # Step 1: Analyze the query
            print("Step 1: Analyzing query...")
            analyzer_result = await safe_agent_run(self.query_analyzer, user_query)
            
            # Extract the structured query information
            try:
                query_info = json.loads(analyzer_result.content)
                print(f"Query analysis complete: {json.dumps(query_info, indent=2)}")
            except json.JSONDecodeError:
                # If the response is not valid JSON, use a simplified approach
                query_info = {"search_terms": user_query}
                print("Could not parse analyzer response as JSON. Using simplified query info.")
            
            # Step 2: Retrieve relevant data with RAG
            print("Step 2: Retrieving data with RAG...")
            retriever_message = f"Search for information related to: {json.dumps(query_info)}"
            retriever_result = await safe_agent_run(self.data_retriever, retriever_message)
            
            # Step 3: Generate the response
            print("Step 3: Generating response...")
            generator_message = f"""
            Original query: {user_query}
            
            Retrieved information: {retriever_result.content}
            
            Please generate a comprehensive response to the original query based on this information.
            """
            response_result = await safe_agent_run(self.response_generator, generator_message)
            
            print("Query processing complete")
            return response_result.content
            
        except Exception as e:
            error_message = f"Error processing query: {str(e)}"
            print(error_message)


In [ ]:
def setup_rag_workflow(collection):
    """
    Set up the RAG-enabled agent workflow
    
    Args:
        collection: ChromaDB collection for vector search
        
    Returns:
        RAGEnabledWorkflow: Configured RAG-enabled agent workflow
    """
    return RAGEnabledWorkflow(collection)

async def main():
    """
    Main function to run the RAG-enabled agent workflow
    """
    # Load the vector database
    print("Loading vector database...")
    collection = load_vector_db()
    
    # Set up the RAG-enabled agent workflow
    print("Setting up RAG-enabled agent workflow...")
    workflow = setup_rag_workflow(collection)
    
    print("RAG-enabled agent workflow ready!")
    print("\nYou can now ask questions about customer visit data.")
    
    # Simple interactive loop for testing
    try:
        while True:
            query = input("\nEnter your query (or 'exit' to quit): ")
            if query.lower() == 'exit':
                break
                
            response = await workflow.process_query(query)
            print("\nResponse:")
            print(response)
    finally:
        # Clean up resources
        await workflow.close()

# Run the main function if this script is executed directly
if __name__ == "__main__":
    main()


In [ ]:
def enhance_context_with_metadata(retrieved_documents):
    """
    Enhance the context with metadata analysis
    
    Args:
        retrieved_documents: List of retrieved documents
        
    Returns:
        str: Enhanced context with metadata insights
    """
    # Extract metadata from documents
    regions = []
    customers = []
    product_divisions = []
    dates = []
    
    for doc in retrieved_documents:
        metadata = doc.get("metadata", {})
        if "region" in metadata and metadata["region"]:
            regions.append(metadata["region"])
        if "customer" in metadata and metadata["customer"]:
            customers.append(metadata["customer"])
        if "product_division" in metadata and metadata["product_division"]:
            product_divisions.append(metadata["product_division"])
        if "visit_date" in metadata and metadata["visit_date"]:
            dates.append(metadata["visit_date"])
    
    # Generate insights from metadata
    insights = []
    
    # Region insights
    if regions:
        region_counts = {}
        for region in regions:
            region_counts[region] = region_counts.get(region, 0) + 1
        most_common_region = max(region_counts.items(), key=lambda x: x[1])[0]
        insights.append(f"Most relevant region: {most_common_region}")
    
    # Customer insights
    if customers:
        customer_counts = {}
        for customer in customers:
            customer_counts[customer] = customer_counts.get(customer, 0) + 1
        most_common_customers = sorted(customer_counts.items(), key=lambda x: x[1], reverse=True)[:3]
        if most_common_customers:
            insights.append(f"Most relevant customers: {', '.join([c[0] for c in most_common_customers])}")
    
    # Product division insights
    if product_divisions:
        division_counts = {}
        for division in product_divisions:
            division_counts[division] = division_counts.get(division, 0) + 1
        most_common_divisions = sorted(division_counts.items(), key=lambda x: x[1], reverse=True)[:3]
        if most_common_divisions:
            insights.append(f"Most relevant product divisions: {', '.join([d[0] for d in most_common_divisions])}")
    
    # Date insights
    if dates:
        try:
            parsed_dates = [dateparser.parse(date) for date in dates if date]
            valid_dates = [date for date in parsed_dates if date]
            if valid_dates:
                earliest_date = min(valid_dates)
                latest_date = max(valid_dates)
                insights.append(f"Date range: {earliest_date.strftime('%Y-%m-%d')} to {latest_date.strftime('%Y-%m-%d')}")
        except:
            pass
    
    # Combine insights
    if insights:
        return "\n".join(insights)
    else:
        return "No additional insights available from metadata."


In [ ]:
def augment_context(self, query, retrieved_data):
    """
    Augment the query with context from retrieved documents
    
    Args:
        query: The original query
        retrieved_data: Data retrieved from the vector database
        
    Returns:
        str: Augmented context for the LLM
    """
    if not retrieved_data or "error" in retrieved_data:
        return f"Query: {query}\n\nNo relevant information found in the database."
    
    # Extract the retrieved documents
    documents = retrieved_data.get("retrieved_data", [])
    if not documents:
        return f"Query: {query}\n\nNo relevant information found in the database."
    
    # Build the context
    context = f"Query: {query}\n\n"
    
    # Add metadata insights
    metadata_insights = enhance_context_with_metadata(documents)
    context += f"Insights from the data:\n{metadata_insights}\n\n"
    
    # Add the retrieved documents
    context += "Relevant information from customer visit database:\n\n"
    
    for i, doc in enumerate(documents):
        context += f"Document {i+1}:\n{doc['text']}\n\n"
    
    return context


In [ ]:
async def initialize_system(df=None):
    """
    Initialize the entire system
    
    Args:
        df: Optional DataFrame with customer visit data
        
    Returns:
        RAGEnabledWorkflow: The initialized workflow
    """
    # Set up the vector database if DataFrame is provided
    if df is not None:
        collection = setup_vector_database(df)
    else:
        # Otherwise, load the existing database
        collection = load_vector_db()
    
    # Set up the RAG-enabled workflow
    workflow = setup_rag_workflow(collection)
    
    return workflow

async def process_user_query(workflow, query):
    """
    Process a user query and return the response
    
    Args:
        workflow: The RAG-enabled workflow
        query: The user query
        
    Returns:
        str: The response to the query
    """
    return await workflow.process_query(query)


# Query Processing Pipeline

In [ ]:
# Import required libraries for the query processing pipeline
import asyncio
import json
import os
from typing import Dict, List, Any, Optional, Union
import pandas as pd
import numpy as np
import re
from datetime import datetime
import dateparser

# For embedding and vector search
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings

# For NLP processing
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# For AutoGen integration
from autogen import AssistantAgent
from autogen import UserMessage
from autogen import OllamaChatCompletionClient


In [ ]:
class QueryProcessor:
    """
    Processes user queries to extract temporal references and key terms
    """
    
    def __init__(self, embedding_model=None):
        """
        Initialize the query processor
        
        Args:
            embedding_model: SentenceTransformer model for embedding generation
        """
        # Initialize NLP components
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()
        
        # Initialize embedding model if not provided
        if embedding_model is None:
            self.embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
        else:
            self.embedding_model = embedding_model
        
        # Temporal reference patterns
        self.temporal_patterns = {
            'date': r'\b\d{1,2}[-/]\d{1,2}[-/]\d{2,4}\b|\b\d{4}[-/]\d{1,2}[-/]\d{1,2}\b',
            'month_year': r'\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)[,\s]+\d{4}\b',
            'month': r'\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\b',
            'year': r'\b\d{4}\b',
            'quarter': r'\bQ[1-4]\s+\d{4}\b|\b(?:first|second|third|fourth)\s+quarter\b',
            'relative': r'\blast\s+(?:week|month|year|quarter)\b|\bnext\s+(?:week|month|year|quarter)\b|\bprevious\s+(?:week|month|year|quarter)\b|\bcurrent\s+(?:week|month|year|quarter)\b'
        }
    
    def extract_temporal_references(self, query):
        """
        Extract temporal references from the query
        
        Args:
            query: The user query
            
        Returns:
            dict: Extracted temporal references
        """
        temporal_refs = {}
        
        # Extract dates and time periods
        for ref_type, pattern in self.temporal_patterns.items():
            matches = re.findall(pattern, query, re.IGNORECASE)
            if matches:
                temporal_refs[ref_type] = matches
        
        # Try to parse any dates found
        parsed_dates = []
        current_date = datetime.now()
        
        # Process explicit dates
        if 'date' in temporal_refs:
            for date_str in temporal_refs['date']:
                try:
                    parsed_date = dateparser.parse(date_str)
                    if parsed_date:
                        parsed_dates.append(parsed_date)
                except:
                    pass
        
        # Process month and year combinations
        if 'month_year' in temporal_refs:
            for month_year in temporal_refs['month_year']:
                try:
                    parsed_date = dateparser.parse(month_year)
                    if parsed_date:
                        parsed_dates.append(parsed_date)
                except:
                    pass
        
        # Process relative time references
        if 'relative' in temporal_refs:
            for rel_time in temporal_refs['relative']:
                try:
                    parsed_date = dateparser.parse(rel_time, settings={'RELATIVE_BASE': current_date})
                    if parsed_date:
                        parsed_dates.append(parsed_date)
                except:
                    pass
        
        # Add parsed dates to the result
        if parsed_dates:
            temporal_refs['parsed_dates'] = [date.strftime('%Y-%m-%d') for date in parsed_dates]
        
        return temporal_refs
    
    def extract_key_terms(self, query):
        """
        Extract key terms from the query
        
        Args:
            query: The user query
            
        Returns:
            list: Extracted key terms
        """
        # Tokenize the query
        tokens = word_tokenize(query.lower())
        
        # Remove stopwords and punctuation
        filtered_tokens = [self.lemmatizer.lemmatize(token) for token in tokens 
                          if token.isalnum() and token not in self.stop_words]
        
        # Extract potential entity names (capitalized words)
        potential_entities = []
        for match in re.finditer(r'\b[A-Z][a-zA-Z]*(?:\s+[A-Z][a-zA-Z]*)*\b', query):
            potential_entities.append(match.group())
        
        # Combine filtered tokens and potential entities
        key_terms = list(set(filtered_tokens + potential_entities))
        
        return key_terms
    
    def vectorize_query(self, query):
        """
        Generate embedding vector for the query
        
        Args:
            query: The user query
            
        Returns:
            np.ndarray: Query embedding vector
        """
        return self.embedding_model.encode(query)
    
    def process_query(self, query):
        """
        Process the user query to extract information and generate embedding
        
        Args:
            query: The user query
            
        Returns:
            dict: Processed query information
        """
        # Extract temporal references
        temporal_refs = self.extract_temporal_references(query)
        
        # Extract key terms
        key_terms = self.extract_key_terms(query)
        
        # Generate query embedding
        query_embedding = self.vectorize_query(query)
        
        # Combine all information
        processed_query = {
            "original_query": query,
            "temporal_references": temporal_refs,
            "key_terms": key_terms,
            "embedding": query_embedding
        }
        
        return processed_query


In [ ]:
class QueryPipeline:
    """
    End-to-end pipeline for processing user queries and generating responses
    """
    
    def __init__(self, collection, embedding_model=None):
        """
        Initialize the query pipeline
        
        Args:
            collection: ChromaDB collection for vector search
            embedding_model: SentenceTransformer model for embedding generation
        """
        # Initialize embedding model
        if embedding_model is None:
            self.embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
        else:
            self.embedding_model = embedding_model
        
        # Initialize query processor
        self.query_processor = QueryProcessor(embedding_model=self.embedding_model)
        
        # Store collection
        self.collection = collection
        
        # Initialize Ollama client for LLM
        self.model_client = OllamaChatCompletionClient(
            model="qwen2.5-coder:0.5b",  # Or any other model you have in Ollama
            temperature=0.7
        )
        
        # Create the response generator agent
        self.response_agent = self.create_response_generator_agent()
        
    async def generate_response(self, augmented_context):
        """
        Generate a response using the LLM with augmented context
        
        Args:
            augmented_context: The context-augmented query
            
        Returns:
            str: Generated response
        """
        try:
            task_result = await self.response_agent.run(task=augmented_context)

            # Preferred: use last_message if available
            if hasattr(task_result, "last_message"):
                return task_result.last_message.content
            
            # Fallback: pull the last TextMessage from .messages
            if hasattr(task_result, "messages") and task_result.messages:
                # assume the last message is the assistant’s answer
                return task_result.messages[-1].content

            # In the unlikely case neither attribute exists
            return str(task_result)
        except Exception as e:
            return f"Error generating response: {str(e)}"

    
    def create_response_generator_agent(self):
        """
        Create the response generator agent
        
        Returns:
            AssistantAgent: The response generator agent
        """
        system_message = """You are a Response Generator Agent specialized in creating informative responses about customer visit data.
        Your task is to:
        1. Use the retrieved data to create a comprehensive response
        2. Structure the information in a clear, readable format
        3. Highlight key insights and patterns
        4. Provide direct answers to the original query
        5. Include relevant details from the visit reports
        
        Make your responses:
        - Concise but informative
        - Well-structured with headings when appropriate
        - Focused on answering the original question
        - Supported by data from the retrieved documents
        
        If the data is insufficient to answer the query completely, acknowledge the limitations.
        """
        
        return AssistantAgent(
            name="ResponseGenerator",
            model_client=self.model_client,
            system_message=system_message
        )
    
    async def search_vector_db(self, processed_query, n_results=5):
        """
        Search the vector database using the processed query
        
        Args:
            processed_query: The processed query information
            n_results: Number of results to retrieve
            
        Returns:
            dict: Search results
        """
        try:
            # Extract query embedding
            query_embedding = processed_query["embedding"].tolist()
            
            # Extract temporal references for filtering
            temporal_refs = processed_query["temporal_references"]
            
            # Prepare filter criteria if temporal references exist
            filter_criteria = None
            if temporal_refs and "parsed_dates" in temporal_refs:
                # This is a simplified example - you would need to adapt this
                # to your specific database schema and filtering capabilities
                pass
            
            # Perform the vector search
            results = self.collection.query(
                query_embeddings=[query_embedding],
                n_results=n_results,
                where=filter_criteria
            )
            
            docs   = results["documents"][0]
            metas  = results.get("metadatas", [[]])[0]
            ids    = results.get("ids", [[]])[0]
            dists  = results.get("distances", [[]])[0]
            
            # Format the results
            formatted_results = []
            if results and 'documents' in results and results['documents']:
                for doc, meta, _id, dist in zip(docs, metas, ids, dists):
                    # Get metadata if available
                    metadata = {}
                    if 'metadatas' in results and results['metadatas'] and i < len(results['metadatas'][0]):
                        metadata = results['metadatas'][0][i]
                    
                    # Format the entry
                    formatted_results.append({
                        "text": doc,
                        "metadata": metadata,
                        "distance": results['distances'][0][i] if 'distances' in results else None,
                        "id": _id,
                        "text": doc,
                        "distance": dist,
                        "metadata": metadata   
                    })
            
            return {
                "retrieved_data": formatted_results,
                "metadata": {
                    "total_found": len(formatted_results),
                    "query": processed_query["original_query"]
                }
            }
        except Exception as e:
            print(f"Error in vector search: {str(e)}")
            return {
                "error": f"Error retrieving data: {str(e)}",
                "retrieved_data": [],
                "metadata": {
                    "total_found": 0,
                    "query": processed_query["original_query"]
                }
            }
    
    def augment_context(self, processed_query, search_results):
        """
        Augment the context with search results and query information
        
        Args:
            processed_query: The processed query information
            search_results: The search results from vector database
            
        Returns:
            str: Augmented context
        """
        if not search_results or "error" in search_results:
            return f"Query: {processed_query['original_query']}\n\nNo relevant information found in the database."
        
        # Extract the retrieved documents
        documents = search_results.get("retrieved_data", [])
        if not documents:
            return f"Query: {processed_query['original_query']}\n\nNo relevant information found in the database."
        
        # Build the context
        context = f"Query: {processed_query['original_query']}\n\n"
        
        # Add temporal references if available
        temporal_refs = processed_query.get("temporal_references", {})
        if temporal_refs:
            context += "Temporal References in Query:\n"
            for ref_type, refs in temporal_refs.items():
                if ref_type != "parsed_dates":
                    context += f"- {ref_type}: {', '.join(refs)}\n"
            if "parsed_dates" in temporal_refs:
                context += f"- Parsed Dates: {', '.join(temporal_refs['parsed_dates'])}\n"
            context += "\n"
        
        # Add key terms if available
        key_terms = processed_query.get("key_terms", [])
        if key_terms:
            context += f"Key Terms in Query: {', '.join(key_terms)}\n\n"
        
        # Add the retrieved documents
        context += "Relevant information from customer visit database:\n\n"
        
        for i, doc in enumerate(documents):
            context += f"Document {i+1}:\n{doc['text']}\n\n"
            if "metadata" in doc and doc["metadata"]:
                context += "Metadata:\n"
                for key, value in doc["metadata"].items():
                    context += f"- {key}: {value}\n"
                context += "\n"
        
        return context

    
    async def process(self, query):
        """
        Process a user query through the entire pipeline
        
        Args:
            query: The user query
            
        Returns:
            str: Generated response
        """
        try:
            print(f"Processing query: {query}")
            
            # Step 1: Process the query
            print("Step 1: Processing query...")
            processed_query = self.query_processor.process_query(query)
            
            # Step 2: Search the vector database
            print("Step 2: Searching vector database...")
            search_results = await self.search_vector_db(processed_query)
                       
            try:            
                # DEBUG: print what we got
                print("\nRetrieved hits:")
                for hit in search_results["retrieved_data"]:  
                    print(f" • ID={hit['id']}, dist={hit['distance']:.4f}, customer={hit['metadata']['customer']}")
                    print(f"   → {hit['text'][:100]}…\n")
            except:
                pass
            
            # Step 3: Augment the context
            print("Step 3: Augmenting context...")
            augmented_context = self.augment_context(processed_query, search_results)
            
            # Step 4: Generate the response
            print("Step 4: Generating response...")
            response = await self.generate_response(augmented_context)
            
            print("Query processing complete")
            return response
            
        except Exception as e:
            error_message = f"Error processing query: {str(e)}"
            print(error_message)
            return f"I encountered an error while processing your query: {str(e)}"
    
    async def close(self):
        """Close the model client connection"""
        await self.model_client.close()


In [ ]:
def setup_query_pipeline(collection):
    """
    Set up the query processing pipeline
    
    Args:
        collection: ChromaDB collection for vector search
        
    Returns:
        QueryPipeline: Configured query pipeline
    """
    return QueryPipeline(collection)

async def main():
    """
    Main function to run the query processing pipeline
    """
    # Load the dataset
    print("Loading dataset...")
    df = pd.read_csv("customer_visits.csv")
    print(f"Dataset loaded with {len(df)} rows")
    
    # Set up the embedding model
    print("Loading embedding model...")
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
    
    # Set up the vector database
    print("Setting up vector database...")
    chroma_path = os.getenv("CHROMADB_PATH", "./chroma_db")
    client = chromadb.Client(Settings(persist_directory=chroma_path))
    
    # Create embedding function for ChromaDB
    class SentenceTransformerEmbedding:
        def __init__(self, model):
            self.model = model
            
        def __call__(self, input):
            embeddings = self.model.encode(input)
            return embeddings.tolist()
    
    embedding_function = SentenceTransformerEmbedding(embedding_model)
    
    # Check if collection exists, otherwise create and populate it
    try:
        collection = client.get_collection(
            name="customer_visits",
            embedding_function=embedding_function
        )
        print(f"Loaded existing collection with {collection.count()} documents")
    except:
        print("Collection not found. Creating and populating...")
        collection = client.create_collection(
            name="customer_visits",
            embedding_function=embedding_function
        )
        
        # Prepare documents for insertion
        documents = []
        metadatas = []
        ids = []
        
        for idx, row in df.iterrows():
            # Create a comprehensive document from the row data
            document = f"""
            Visit Date: {row['Visit Plan: Visit Date']}
            Created Date: {row['Visit Report: Created Date']}
            Region: {row['Visit Plan: Owner Region']}
            Owner Email: {row['Visit Plan: Visit Owner Email']}
            Owner Name: {row['Visit Plan: Owner Name']}
            Customer: {row['Customer']}
            Customer SAP Code: {row['Customer SAP Code']}
            Product Division: {row['Visit Plan: Product Division']}
            Next Steps: {row['Next Steps']}
            Outcome of meeting: {row['Outcome of meeting']}
            """
            
            # Create metadata for filtering
            metadata = {
                "visit_date": str(row['Visit Plan: Visit Date']),
                "created_date": str(row['Visit Report: Created Date']),
                "region": row['Visit Plan: Owner Region'],
                "owner_email": row['Visit Plan: Visit Owner Email'],
                "owner_name": row['Visit Plan: Owner Name'],
                "customer": row['Customer'],
                "customer_sap_code": str(row['Customer SAP Code']),
                "product_division": row['Visit Plan: Product Division']
            }
            
            documents.append(document)
            metadatas.append(metadata)
            ids.append(f"doc_{idx}")
        
        # Add documents to the collection in batches
        batch_size = 100
        for i in range(0, len(documents), batch_size):
            end_idx = min(i + batch_size, len(documents))
            collection.add(
                documents=documents[i:end_idx],
                metadatas=metadatas[i:end_idx],
                ids=ids[i:end_idx]
            )
            print(f"Added documents {i} to {end_idx-1}")
        
        print(f"Vector database setup complete with {len(documents)} documents")
    
    # Set up the query pipeline
    print("Setting up query pipeline...")
    pipeline = setup_query_pipeline(collection)
    
    print("Query pipeline ready!")
    print("\nYou can now ask questions about customer visit data.")
    
    # Simple interactive loop for testing
    try:
        while True:
            query = input("\nEnter your query (or 'exit' to quit): ")
            if query.lower() == 'exit':
                break
                
            response = await pipeline.process(query)
            print("\nResponse:")
            print(response)
    finally:
        # Clean up resources
        await pipeline.close()

# Run the main function if this script is executed directly
if __name__ == "__main__":
    main()


In [ ]:
# 1. Imports and async helper
import asyncio
from sentence_transformers import SentenceTransformer


async def run_single_query(query: str):
    
    # SentenceTransformer embedding model must match the one used for ingest
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
    # Wrap it in Chroma’s embedding interface
    class EmbeddingFn:
        def __init__(self, model): self.model = model
        def __call__(self, texts): return self.model.encode(texts).tolist()
    
    # 3. Instantiate your pipeline
    pipeline = QueryPipeline(collection, embedding_model=embedding_model)
    
    # 4. Process the query and print the answer
    response = await pipeline.process(query)
    print("\n=== Response ===\n", response)
    
    # 5. Clean up
    await pipeline.close()

# 6. Run it!
query_text = "What next steps were planned for Ganges International Pvt Ltd?"
asyncio.run(run_single_query(query_text))
